In [1]:
import requests
from bs4 import BeautifulSoup

def get_rotowire_schedule(today=True):
    if today:
        url = "https://www.rotowire.com/baseball/daily-lineups.php"
    else:
        url = "https://www.rotowire.com/baseball/daily-lineups.php?date=tomorrow"

    response = requests.get(url)
    
    if response.status_code == 200:
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        lineup_boxes = soup.find_all("div", class_="lineup__box")
        game_times = soup.find_all("div", class_="lineup__time")
        return lineup_boxes, game_times
        #for box in lineup_boxes:
        #    content = box.get_text(strip=True)
        #    print(content)
    else:
        print(f"Failed to fetch webpage. Status code: {response.status_code}")


games, game_times = get_rotowire_schedule()


from importlib import reload
from src.simulation import Game as g
from src.simulation import Team as t
reload(g)
reload(t)

g.Game(games[0], game_times[0].get_text(strip=True) )

2024-05-24 16:08:14,841 - src.simulation.Game - INFO - Fitting home team models
2024-05-24 16:08:14,844 - src.simulation.Game - INFO - Fitting away team models
2024-05-24 16:08:14,846 - src.simulation.Game - INFO - Init complete for TOR @ DET starting at 6:40 PM ET


6:40 PM ET
Comerica Park 2394
TOR DET


In [7]:
game_times[0].get_text(strip=True)

'12:35 PM ET'

In [2]:
from importlib import reload
from src.simulation import Game as g
from src.simulation import Team as t
reload(g)
reload(t)

<module 'src.simulation.Team' from '/home/reece/miniconda3/envs/mlb/lib/python3.12/site-packages/src/simulation/Team.py'>

In [3]:

games[0]
game_times[0]
g.Game(games[0], game_times[0].get_text(strip=True) )

2024-05-23 13:59:36,421 - src.simulation.Game - INFO - Fitting home team models
2024-05-23 13:59:36,422 - src.simulation.Game - INFO - Fitting away team models
2024-05-23 13:59:36,422 - src.simulation.Game - INFO - Init complete for SEA @ NYY starting at 12:35 PM ET


12:35 PM ET
None None
SEA NYY


In [6]:
t.Team(games[0], True)

147
NYY


In [6]:
home_team = t.Team(games[0], is_home=True)
away_team = t.Team(games[0], is_home=False)

In [13]:
#home_team.print_team_info()
home_team.name
away_team.logo

'https://content.rotowire.com/images/teamlogo/baseball/100SEA.png?v=6'

In [17]:
def get_rotowrite_info(game, team='home'):

    # get lineup boxes
    lineup_soup = game.find("div", class_=f"lineup__team is-{team}")
    lineup_list_soup = game.find("ul", class_=f"lineup__list is-{team}")

    # get team name, logo, and pitcher
    team_name = lineup_soup.find("div", class_="lineup__abbr").get_text(strip=True)
    team_logo = lineup_soup.find("img")["src"]
    pitcher = lineup_list_soup.find("li", class_="lineup__player-highlight").find("a")["href"].split("-")[-1]

    #get see if lineup is expected or confirmed
    is_confirmed = lineup_list_soup.find("li", class_='lineup__status is-confirmed') 
    is_expected = lineup_list_soup.find("li", class_='lineup__status is-expected')
    if is_confirmed:
        lineup_confirmed = True
    else:
        lineup_confirmed = False

    # get batters
    batters = []
    batters_soup = game.find("ul", class_=f"lineup__list is-{team}").find_all("li", class_="lineup__player")
    for batter in batters_soup:
        batter_rotowireid = batter.find("a")["href"].split("-")[-1]
        batters.append(batter_rotowireid)
        
    return team_name, team_logo, pitcher, batters
    
team_name, team_logo, pitcher, batters, lineup_confirmed = get_rotowrite_info(boxes[5], team='visit')

None
<li class="lineup__status is-expected">
<div class="dot is-medium is-yellow" style="margin-right:5px;"></div>Expected Lineup
            </li>


In [21]:
from importlib import reload
from src.simulation import Player as p
reload(p)

pitch = p.Player(rotowire_id=float(pitcher))
pitch.print_info()

batters_cl = []
for batter in batters:
    batters_cl.append(p.Player(rotowire_id=float(batter)))

for batter in batters_cl:
    batter.print_info()

Luis Castillo, SEA, P (mlbid: 622491, rotowireid: 14495.0)
J.P. Crawford, SEA, SS (mlbid: 641487, rotowireid: 12954.0)
Julio Rodriguez, SEA, OF (mlbid: 677594, rotowireid: 15064.0)
Jorge Polanco, SEA, 2B (mlbid: 593871, rotowireid: 11359.0)
Luke Raley, SEA, OF (mlbid: 670042, rotowireid: 15646.0)
Cal Raleigh, SEA, C (mlbid: 663728, rotowireid: 15518.0)
Mitch Haniger, SEA, OF (mlbid: 571745, rotowireid: 12491.0)
Dominic Canzone, SEA, OF (mlbid: 686527, rotowireid: 16173.0)
Ty France, SEA, 1B (mlbid: 664034, rotowireid: 15038.0)
Josh Rojas, SEA, 2B (mlbid: 668942, rotowireid: 15913.0)


In [5]:
lineup_boxes[0]

<div class="lineup__box">
<div class="lineup__top has-bad-weather">
<div class="lineup__teams">
<div class="lineup__team is-visit">
<img alt="SEA" class="lineup__logo" src="https://content.rotowire.com/images/teamlogo/baseball/100SEA.png?v=6"/>
<div class="lineup__abbr">SEA</div>
</div>
<div class="lineup__team is-home">
<img alt="NYY" class="lineup__logo" src="https://content.rotowire.com/images/teamlogo/baseball/100NYY.png?v=6"/>
<div class="lineup__abbr">NYY</div>
</div>
</div>
</div>
<a class="lineup__matchup" href="/baseball/box-score/yankees-vs-mariners-2024-05-23-2618430">
<div class="lineup__mteam is-visit">
                                        Mariners                                        <span class="lineup__wl">(27-23)</span>
</div>
<div class="lineup__mteam is-home">
                                        Yankees                                        <span class="lineup__wl">(34-17)</span>
</div>
</a>
<div class="lineup__main">
<ul class="lineup__list is-visit">
<li 